In [1]:
import ollama
ollama.pull("llama3:8b")

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [ ]:
# --- Classification Function ---
def classify_urdu_question_gemma(urdu_question):
    prompt = f"""You are given a question in Urdu. Classify it as SINGLEHOP or MULTIHOP.

Definitions:
- SINGLEHOP: Can be answered using one fact, sentence, or document.
- MULTIHOP: Requires combining multiple facts or reasoning over multiple steps.

Examples:

سوال: "پاکستان کے وزیر اعظم کا نام کیا ہے؟"
جواب: SINGLEHOP

سوال: "جارج واشنگٹن کی پیدائش کے وقت امریکہ کا صدر کون تھا؟"
جواب: MULTIHOP

سوال: "ایپل کمپنی کے شریک بانی نے کہاں تعلیم حاصل کی، اور اس یونیورسٹی کا مشہور فارغ التحصیل کون ہے؟"
جواب: MULTIHOP

سوال: "یہ فلم 2010 میں ریلیز ہوئی اور سکاٹ لینڈ میں 1828 کے قتلوں پر مبنی ہے۔"
جواب: SINGLEHOP

---

Now classify the following:

سوال: "{urdu_question}"
جواب:"""


    try:
        response = ollama.chat(
            model='gemma3:4b',
            messages=[{"role": "user", "content": prompt}]
        )
        reply = response['message']['content'].strip().lower()

        if 'multihop' in reply:
            return 'multihop'
        elif 'singlehop' in reply or 'simple' in reply:
            return 'singlehop'

        print(f"⚠️ Unexpected response: {reply}")
        return "unknown"

    except Exception as e:
        print(f"❌ Error processing question: {urdu_question}\n↪ {e}")
        return "error"


In [3]:
def decompose_urdu_query(urdu_query: str) -> dict:
    """Returns dictionary with q1 and q2 keys containing sub-questions"""
    refined_prompt = f"""
**Role**: You are an expert Urdu linguistic analyst specializing in question decomposition. Your task is to break down complex Urdu questions into their fundamental components.

**Task Instructions**:
1. Carefully analyze the given Urdu question to identify its core components
2. Extract exactly 2 sub-questions that:
   - Are necessary to answer the main question
   - Cover distinct aspects of the problem
   - Have clear logical progression (answer to q1 helps answer q2)
3. Both sub-questions must:
   - Be in proper Urdu language
   - Be grammatically correct
   - Be clear and concise
   - Use relevant domain terminology

**Output Format Requirements**:
- Use EXACTLY this format:
  q1: [پہلا ذیلی سوال]
  q2: [دوسرا ذیلی سوال]
- Each sub-question must be on a new line
- Do not include any additional commentary or explanation
- Do not number the questions (use only q1:/q2: prefixes)

**Example 1**:
Input: اگر لاہور میں فضائی آلودگی کی سطح دہلی سے زیادہ ہے اور فضائی آلودگی پھیپھڑوں کے کینسر کا سبب بن سکتی ہے، تو لاہور کے رہائشیوں کو کس قسم کے طبی چیک اپ کروانے چاہئیں؟
Output:
q1: لاہور اور دہلی میں فضائی آلودگی کی سطح کا موازنہ کیا ہے؟
q2: فضائی آلودگی پھیپھڑوں کے کینسر کا سبب کیسے بنتی ہے؟

**Example 2**:
Input: اگر کراچی میں بجلی کے نرخ 30% بڑھ گئے ہیں اور یہ صنعتوں کو متاثر کر رہا ہے، تو حکومت کو کون سی سبسڈیاں دینی چاہئیں؟
Output:
q1: کراچی میں بجلی کے نرخوں میں اضافے کی موجودہ شرح کیا ہے؟
q2: بجلی کے مہنگے ہونے سے صنعتوں پر کس قسم کے اثرات مرتب ہو رہے ہیں؟

**Current Task**:
Input: {urdu_query}
Output:
"""
    
    try:
        response = ollama.generate(
            model='llama3:8b',
            prompt=refined_prompt,
            options={
                'temperature': 0.5,
                'num_ctx': 2048
            }
        )
        
        output = response['response'].strip()
        
        result = {}
        for line in output.split('\n'):
            line = line.strip()
            if line.startswith('q1:'):
                result['q1'] = line[3:].strip()
            elif line.startswith('q2:'):
                result['q2'] = line[3:].strip()
        
        return result if len(result) == 2 else {}
    
    except Exception as e:
        print(f"Decomposition error: {str(e)}")
        return {}


In [4]:
def query_context_relevance_check(query_urdu: str, context_urdu: str) -> bool:
    prompt = f"""
You are a binary classifier.

Your task is to decide whether the following Urdu *context* is relevant to the Urdu *question*. You must answer ONLY with **True** or **False** — no explanation, no commentary, just one word: True or False.

Criteria:
- If the context helps answer the question directly or indirectly, reply: True
- If the context is unrelated, confusing, or insufficient, reply: False

IMPORTANT:
- Do NOT explain your answer.
- Do NOT include any additional comments.
- Just respond with: True or False

---

Question (Urdu): {query_urdu}

Context (Urdu): {context_urdu}

Answer (True/False):
"""

    try:
        response = ollama.chat(
            model="llama3:8b",
            messages=[{"role": "user", "content": prompt}]
        )
        answer = response['message']['content'].strip().lower()
        return answer == 'true'
    except Exception as e:
        print(f"Error during relevance check: {e}")
        return False


In [ ]:
from sentence_transformers import SentenceTransformer
import pickle
import faiss
import os
import torch


def load_retriever(
    index_path: str,
    chunks_path: str
):
    # Initialize device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Load SentenceTransformer model (E5-large)
    model = SentenceTransformer("intfloat/e5-large", device=device)
    
    # Configure for Urdu text
    model.max_seq_length = 512  # Set based on your earlier analysis
    model.tokenizer.do_lower_case = False  # Preserve Urdu characters
    
    # Load FAISS index
    index = faiss.read_index(index_path)
    
    # Load stored chunks
    with open(chunks_path, "rb") as f:
        chunks_list = pickle.load(f)
    
    return model, index, chunks_list, device



c:\hammad workings\Thesis\Multihop for Urdu\Multihop for Urdu\code\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model, index, chunks_list, device = load_retriever(
    index_path="../../vector_db/paragraphs/5884_paras/5884_paras_faiss_index.index",
    chunks_path="../../data_storage/Paragraph_chunks/5884_paragraphs/5884_chunks.pkl"
)

In [ ]:
def retrieve_documents(query,k=3):
   
    query_embedding = model.encode(
        [query],
        convert_to_tensor=False,  # Return numpy array for FAISS
        normalize_embeddings=True,
        show_progress_bar=False
    )
    query_embedding = model.encode(
        [query],
        convert_to_tensor=False,
        normalize_embeddings=True,
        show_progress_bar=False
    )
    
    # Search FAISS index
    _, indices = index.search(query_embedding, k)  # Dummy variable _ for distances
    
    # Return only the chunks
    return [chunks_list[i] for i in indices[0]]


In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def expand_multihop_context(intermediate_ctx, query, q1, q2, k):
    try:
        combined_query = q1 + intermediate_ctx + q2
        second_hop_contexts = retrieve_documents(combined_query, k)

        relevant_contexts = []

        with ThreadPoolExecutor() as inner_executor:
            futures = [
                inner_executor.submit(query_context_relevance_check, query, ctx)
                for ctx in second_hop_contexts
            ]

            for i, future in enumerate(as_completed(futures)):
                try:
                    if future.result():
                        relevant_contexts.append(second_hop_contexts[i])
                except Exception as e:
                    print("Error during relevance check:", e)

        return relevant_contexts

    except Exception as e:
        print("Error in expand_multihop_context:", e)
        return []


def get_context_of_multihop(query, type, model=model, index=index, chunks_list=chunks_list, k=3):
    # Step 1: Classification
    start_classification = time.time()
    classification = classify_urdu_question_gemma(query)
    classification_time = time.time() - start_classification

    if type == "easy":
        decomposition_time = 0.0
        start_retrieval = time.time()
        context = retrieve_documents(query, model, index, chunks_list, k)
        retrieval_time = time.time() - start_retrieval
        return context, classification, classification_time, decomposition_time, retrieval_time

    else:
        # Step 2: Decomposition
        start_decomposition = time.time()
        decomposition = decompose_urdu_query(query)
        q1 = decomposition.get("q1", "")
        q2 = decomposition.get("q2", "")
        decomposition_time = time.time() - start_decomposition

        # Step 3: First-hop retrieval for q1 and q2
        start_retrieval = time.time()
        context_q1 = retrieve_documents(q1, k)
        context_q2 = retrieve_documents(q2, k)

        # Combine base context
        main_context = context_q1 + context_q2

        additional_contexts = []

        # Expand q1
        with ThreadPoolExecutor() as executor_q1:
            futures_q1 = [
                executor_q1.submit(expand_multihop_context, ctx, query, q1, q2, k)
                for ctx in context_q1[:k]
            ]
            for future in as_completed(futures_q1):
                result = future.result()
                additional_contexts.extend(result)

        # Expand q2
        with ThreadPoolExecutor() as executor_q2:
            futures_q2 = [
                executor_q2.submit(expand_multihop_context, ctx, query, q1, q2, k)
                for ctx in context_q2[:k]
            ]
            for future in as_completed(futures_q2):
                result = future.result()
                additional_contexts.extend(result)

        # Merge all retrieved content
        main_context.extend(additional_contexts)
        retrieval_time = time.time() - start_retrieval

        return main_context, classification, classification_time, decomposition_time, retrieval_time


In [9]:
import ollama
ollama.pull('hf.co/large-traversaal/Alif-1.0-8B-Instruct:f16')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [ ]:
import ollama

def generate_using_llama3(context, query):
    prompt = f"""You are a helpful assistant. You will be given a context and a question, both in Urdu.
Your task is to answer the question using the context only. 
Your answer should be **clear, concise, and entirely in Urdu**.
Context:
{context}

Question:
{query}

Answer in Urdu:"""

    try:
        response = ollama.chat(
            model='llama3:8b',
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response['message']['content'].strip()
    except Exception as e:
        print("Error during generation:", e)
        return "جواب پیدا کرنے میں خرابی ہوئی۔"

import ollama

def generate_using_alif(context, query, alif_model='hf.co/large-traversaal/Alif-1.0-8B-Instruct:f16'):


    prompt = f"""آپ کو ایک سوال اور اس سے متعلق ایک سیاق و سباق دیا گیا ہے۔ براہ کرم سیاق و سباق کا بغور مطالعہ کریں اور اسی کی بنیاد پر درست، مختصر اور جامع جواب دیں۔

### سوال:
{query}

### سیاق و سباق:
{context}

### جواب:
"""

    response = ollama.chat(
        model=alif_model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        stream=False
    )

    return response['message']['content']




def prompt_llama_gemma_balanced_english(query, context):
    return f"""
You are an expert assistant. Please answer the following question in fluent and clear Urdu. If possible, avoid using English words. Do not use bullet points or lists. Write your response in concise paragraphs.

### Question:
{query}

### Context:
{context}

### Answer:
"""

def generate_using_gemma(query, context):
    model_name = "gemma3:4b"
    prompt = prompt_llama_gemma_balanced_english(query, context)

    try:
        response = ollama.chat(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            stream=False
        )
        output = response['message']['content'].strip()
    except Exception as e:
        output = f"Error: {e}"

    return output


In [ ]:
import time

def multihop_handling_modLQR(query, type, model=model, index=index, chunks_list=chunks_list, k=3):
    # Step 1: Get context and timings
    context, classification, classification_time, decomposition_time, retrieval_time = get_context_of_multihop(
        query, type, model=model, index=index, chunks_list=chunks_list, k=k
    )

    # Flatten context if it's a list of strings
    if isinstance(context, list):
        combined_context = "\n".join(context)
    else:
        combined_context = context

    # Step 2: Generate answer and measure time
    start_gen = time.time()
    final_answer = generate_using_gemma(combined_context,query)
    generation_time = time.time() - start_gen

    # Step 3: Compute total time
    total_time = classification_time + decomposition_time + retrieval_time + generation_time

    return {
        "classification": classification,
        "retrieved_context": context,
        "final_answer": final_answer,
        "timings": {
            "classification_time": classification_time,
            "decomposition_time": decomposition_time,
            "retrieval_time": retrieval_time,
            "generation_time": generation_time,
            "total_time": total_time
        }
    }


In [ ]:
import pandas as pd
from tqdm import tqdm
import time

# Load your source CSV
df = pd.read_csv("../../../Dataset_code_csvs/hotpotQA/hotpotQA_dataset_versions/5884paras_598queries/Urdu/598_QnAs_translated.csv")

# Output CSV path
output_path = "../../results/pipeline results/5884paras_598qna/modLQR_processed_results.csv"

# Initialize storage
results = []
batch_times = []
total_start = time.time()

# Loop over each row
for idx, row in tqdm(df.iterrows(), total=len(df)):
    query = row["translated_question"]
    answer = row["translated_answer"]
    question_type = row["level"]

    start_time = time.time()

    try:
        result = multihop_handling_modLQR(query, question_type)

        classification = result["classification"]
        retrieved_context = result["retrieved_context"]
        final_answer = result["final_answer"]
        timings = result["timings"]
        total_time_one = timings["total_time"]

    except Exception as e:
        print(f"Error processing query {idx}: {e}")
        classification = "Error"
        retrieved_context = "Error"
        final_answer = "Error"
        timings = {
            "classification_time": 0,
            "decomposition_time": 0,
            "retrieval_time": 0,
            "generation_time": 0,
            "total_time": 0
        }
        total_time_one = 0

    elapsed = time.time() - start_time
    batch_times.append(elapsed)

    results.append({
        "translated_question": query,
        "translated_answer": answer,
        "classification": classification,
        "retrieved_context": retrieved_context,
        "final_answer": final_answer,
        "classification_time": timings["classification_time"],
        "decomposition_time": timings["decomposition_time"],
        "retrieval_time": timings["retrieval_time"],
        "generation_time": timings["generation_time"],
        "total_time": timings["total_time"],
        "level": question_type
    })

    print(f"Processed record {idx+1}/{len(df)} in {elapsed:.2f} seconds.")

    # Write batch every 100 queries
    if (idx + 1) % 100 == 0:
        pd.DataFrame(results).to_csv(output_path, mode='a', header=not bool(idx), index=False, encoding="utf-8-sig")
        avg_batch_time = sum(batch_times) / len(batch_times)
        print(f"\n--- Saved batch up to record {idx+1}")
        print(f"Average time for last 100 records: {avg_batch_time:.2f} seconds\n")
        results = []
        batch_times = []

# Save any remaining records
if results:
    pd.DataFrame(results).to_csv(output_path, mode='a', header=not bool(len(df) % 100), index=False, encoding="utf-8-sig")

# Final stats
total_elapsed = time.time() - total_start
avg_total_time = total_elapsed / len(df)
print(f"\n✅ All records processed.")
print(f"Total processing time: {total_elapsed:.2f} seconds.")
print(f"Average time per record: {avg_total_time:.2f} seconds.")


  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 98/98 [1:41:15<00:00, 61.99s/it]  

                                 translated_question translated_answer  \
0  سیری بی 2017 - 2017 (اسپانسرشپ کی وجوہات کی بن...         1929ء میں   
1  "آکسفورڈ کالج کے ایک ساتھی ایلک نیلر ڈکن نے کہ...       کرپٹوولوجسٹ   
2  "جراسک پارک کے اداکار ڈیوڈ ہنری ہوانگ نے ""دی ...        بی ڈی وانگ   
3  کون سا کردار ، ڈین کاسٹیلینیٹا کی آواز ، سمپسن...        دادا سمپسن   
4     کون تھا ایک حصہ S#arp، لی Ji-hye یا کرٹس رائٹ؟          لی جی ہے   

  classification                                  retrieved_context  \
0       multihop  [سیری بی (اسپانسرشپ وجوہات کی بناء پر سیری بی ...   
1       multihop  [۔ ریچر اس وقت اسٹینفورڈ یونیورسٹی میں اسٹینفو...   
2      singlehop  [۔ یہ فلم والٹر ونجر نے پروڈیوس کی ہے۔ اس فلم ...   
3       multihop  [۔ اس کے کریڈٹ میں ""نائٹ آف دی لونگ ڈیڈ"" ، "...   
4       multihop  [۔ اس میں ممبر کیم لیپ کا تعارف کیا گیا ہے اور...   

                                        final_answer  classification_time  \
0  سیریز بی ایک فٹ بال لیگ ہے جو اٹلی میں مقامی س..